In [1]:
import undetected_chromedriver.v2 as uc
from selenium.webdriver.common.by import By                                  ### Module
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from multiprocessing import connection
import mysql.connector
import time

In [2]:
driver = uc.Chrome()  # loading undetectable chrome browser

driver.get('https://www.careerguide.com/career-options')  ## scraping careerguide website to get career names

In [3]:
Catogery = []
State = []
Job_links = []
company_links = []
Company_Name = []
Job_location = []
Job_Title= []
Mode = []

In [4]:
#establishing the connection
conn = mysql.connector.connect( user='choudhary', password='choudhary', host='127.0.0.1',port='3306', database='job_data',auth_plugin='mysql_native_password')

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

### categories

In [5]:
categories = []
categoriesOpt = driver.find_elements(By.XPATH,"//*[@class='c-font-bold']/a")
for categorey in categoriesOpt:
    categories.append(categorey.text)
    # Preparing SQL query to INSERT a record into the database.
    sql = """INSERT INTO data_categorie (job_categorey) VALUES (?)", (categorey.text,)"""

    # Executing the SQL command
    cursor.execute("INSERT INTO job_categorie (job_categorey) VALUES (%s)", (categorey.text,))

    # Commit your changes in the database
    conn.commit()

In [6]:
careerTitle = []  # list to store career title
careerTitleOpt= driver.find_elements(By.XPATH,"//*[@class='col-md-4']/ul/li")  ## scraping careerTitle from careerguide website

#### subcategories

In [8]:
checkCatogries = driver.find_elements(By.XPATH,"//*[@class='col-md-4']")

for i in range(1,len(checkCatogries)+1):
    categorie_div = driver.find_element(By.XPATH,f"(//*[@class='col-md-4'])[{i}]")
    categorey = categorie_div.text.split('\n')[0]
    if(categorey == "Institutes in India"):
        pass
    else:
        lst = [categorie_div.text.split('\n')[1:]]

        for subcategorey in lst[0]:
            careerTitle.append(str(subcategorey))
            # Executing the SQL command
            try:
                cursor.execute("INSERT INTO job_subcategorie (job_sub_categorey,job_categorey_id) VALUES (%s,%s)", (str(subcategorey),categorey,))

                # Commit your changes in the database
                conn.commit()
            except:
                pass

In [9]:
driver.get('https://www.linkedin.com/')  ## getting linkedin website to scrape

In [10]:
username = driver.find_element(By.ID,"session_key")              ## signing with username and password on linkedin
username.send_keys("shamshadkrunal2438@gmail.com")           ## sending input text to input area
password = driver.find_element(By.ID,"session_password")
password.send_keys("wce@2438")                           ##****your_password*****

In [11]:
try:
    login_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME,"sign-in-form__submit-button"))   ## locator login button for linkedin
    )
    login_button.click()   # clicking login button
finally:
    print("cannot find Button")

cannot find Button


In [12]:
job_button=driver.find_element(By.XPATH,"(//*[@class='app-aware-link  global-nav__primary-link'])[2]") ## locator for job section
job_button.click()

#### Locations

In [13]:
Locations = ['goa','maharashtra','gujrat','telangana','uttarpardesh','tamilnadu','karnataka']  ## for now only scraping for three location

for state in Locations:
    # Executing the SQL command
    try:
        cursor.execute("INSERT INTO job_state (state) VALUES (%s)", (state,))
        # Commit your changes in the database
        conn.commit()
    except:
        pass

In [14]:
infos=[]
comp_links=[] ## list to store company links to scrape their info later on

In [15]:
## making location, mode , job title, company name from info above scraped due to they are separated by newline only in same locator
def make(info):
    lst = []
    string = ""
    for i in info:
        if i!="\n":
            string+=i
        if i=="\n": 
            lst.append(string)
            string=""
            
    lst.append(string)   
    return lst
    

#### company deatail function to scrape and store data

In [16]:
def company_detail(comp,Title,state):
    url = comp + 'about/'  ## definning url with about section of company
        
    driver.get(url)
    time.sleep(2)
    ## company name locator
    try:
        company_name=driver.find_element(By.XPATH,"//*[@class='block mt2']//h1") 
    except:
        company_name=''
        
    ## locator for company description
    try:
        description=driver.find_element(By.XPATH,"//*[@class='break-words white-space-pre-wrap mb5 text-body-small t-black--light']")
    except:
        description=''
        
    try:
        cursor.execute("INSERT INTO job_companydetail (name,description,job_sub_categorey_id,state_id) VALUES (%s,%s,%s,%s)",
                       (company_name.text,description.text,Title,state))

        # Commit your changes in the database
        conn.commit()
    except:
        pass

##### calling company detail func on for resopective company link

In [17]:
def make_company_details(links):
    for link in links:
        company_detail(link,Title,Location)

#### get job detail function 

In [18]:
def get_jobdetail(info_div):
    for i in info_div:                    
        try:
            infos.append(i.text)   ## appending job info which contains (location,mode,job title,company name) separated by newline
            info = i.text
        except:
            pass

        try:
            ## appendig state location
            State.append(Location)
        except:
            pass


        lst = make(info)  ### calling make correct information function
        
        try:
            ## appending job title
            Job_Title.append(lst[0])
        except:
            Job_Title.append('')

        try:
            ## appending company name
            Company_Name.append(lst[1])
            print(lst[1])
        except:
            Company_Name.append('')

        try:
            ## appending job title 
            Job_location.append(lst[2])
        except:
            Job_location.append()
            
        print(lst)
        
        ## saving to model in django-mysql
        try:
            cursor.execute("INSERT INTO job_jobdetail (job_position,company_id,location,job_sub_categorey_id) VALUES (%s,%s,%s,%s)",
                           (lst[0],lst[1],lst[2],Title,))
            # Commit your changes in the database
            conn.commit()
        except:
            pass

#### scraping location and state wise data

In [19]:
for Location in Locations:  ## double looping for each location with job careertitle
    for Title in careerTitle[12:]:
        url = "https://www.linkedin.com/jobs/search/?currentJobId=3348462787&keywords="+Title+"&location="+Location+"%2C%20India&refresh=true"
        
        driver.get(url)
        
        flag=1
        page_no = 25
        current_url = driver.current_url
            
        while(flag>0):
            ## locator for job info which as location, mode, title, company name separeted by newline
            info_div=driver.find_elements(By.XPATH,"//*[@class='flex-grow-1 artdeco-entity-lockup__content ember-view']")

            ## locator for job links and company links
            job_Links = driver.find_elements(By.XPATH,"(//*[@class='flex-grow-1 artdeco-entity-lockup__content ember-view']/div[1]/a)")
            comp_Links = driver.find_elements(By.XPATH,"(//*[@class='flex-grow-1 artdeco-entity-lockup__content ember-view']/div[2]/a)")

            # try for if there is any job with given title
            try:
                checkProduct=driver.find_element(By.XPATH,"//*[@class='t-24 t-black t-normal text-align-center']").text
            except:
                checkProduct=''

            ## if no job with given title found pass to next job title
            if(checkProduct=="No matching jobs found."):
                flag=0
            else:
                company_links =[]
                for comp_link in comp_Links:
                    
                    try:
                        link = comp_link.get_attribute('href')
                        company_links.append(link)
                    except:
                        pass
            
                make_company_details(company_links)
        
                ## re-entrying url after scraping and storing company data to store job detail 
                ### because company is foreignkey for job detail
                url = "https://www.linkedin.com/jobs/search/?currentJobId=3348462787&geoId=106187582&keywords="+Title+"&location="+Location+"%2C%20India&refresh=true"
                driver.get(url)
                
                ## locator for job info which as location, mode, title, company name separeted by newline
                time.sleep(2)
                info_div=driver.find_elements(By.XPATH,"//*[@class='flex-grow-1 artdeco-entity-lockup__content ember-view']")
                get_jobdetail(info_div)
                
                ## going to next page for more jobs
                next_page_url = current_url + "&start=" + str(page_no)
                page_no=page_no+25
                
                try:
                    checkNextPageButton=driver.find_element(By.XPATH,"//*[@class='jobs-search-results-list__pagination pv5 artdeco-pagination ember-view']").text
                    if(checkNextPageButton):
                        driver.get(next_page_url) 
                except:
                    checkNextPageButton=''
                    flag=0

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=107.0.5304.107)
Stacktrace:
Backtrace:
	Ordinal0 [0x008CACD3+2075859]
	Ordinal0 [0x0085EE61+1633889]
	Ordinal0 [0x0075B7BD+571325]
	Ordinal0 [0x00742E1A+470554]
	Ordinal0 [0x007AAA0B+895499]
	Ordinal0 [0x007BAC96+961686]
	Ordinal0 [0x007A7136+880950]
	Ordinal0 [0x0077FEFD+720637]
	Ordinal0 [0x00780F3F+724799]
	GetHandleVerifier [0x00B7EED2+2769538]
	GetHandleVerifier [0x00B70D95+2711877]
	GetHandleVerifier [0x0095A03A+521194]
	GetHandleVerifier [0x00958DA0+516432]
	Ordinal0 [0x0086682C+1665068]
	Ordinal0 [0x0086B128+1683752]
	Ordinal0 [0x0086B215+1683989]
	Ordinal0 [0x00876484+1729668]
	BaseThreadInitThunk [0x7723FEF9+25]
	RtlGetAppContainerNamedObjectPath [0x77827BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77827B8E+238]


In [ ]:
driver.quit()  ## closing driver